## Pandas DataFrames II


### Programming for Data Science
### Last Updated: Jan 15, 2023
---  


### PREREQUISITES
- variables
- data types
- operators
- list comprehensions (not essential)
- numpy arrays (not essential)
- pandas DataFrames I


### SOURCES 
- ten minutes to pandas  
https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html


- pivot_table()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html


- concat()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html


- merge()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html


- get_dummies()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html



### OBJECTIVES
- Introduce more advanced pandas dataframe operations for data munging
 


### CONCEPTS

- DataFrame
- apply()
- aggregation using split-apply-combine
- pivot_table()
- groupby()
- concat()
- merging/joining dataframes with merge(), concat()
- reshaping data
- dummy coding categorical data


---

## Additional Pandas DataFrame Functionality

Pandas DataFrames I covered creating, modifying, and subsetting DataFrames, among other topics.  
These notes will demonstrate further methods for data munging and analysis.

In [1]:
import numpy as np
import pandas as pd

#### Load Iris

In [10]:
import seaborn as sns

iris = sns.load_dataset('iris')

In [11]:
iris.head(2)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [4]:
import sys
sys.getsizeof(iris)

13564

## I. `apply()`

Apply a transformation to each record. Uses a `lambda` function.

In [5]:
iris['sepal_len_sq'] = iris.sepal_length.apply(lambda x: x**2)
iris.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_len_sq
0,5.1,3.5,1.4,0.2,setosa,26.01
1,4.9,3.0,1.4,0.2,setosa,24.01
2,4.7,3.2,1.3,0.2,setosa,22.09
3,4.6,3.1,1.5,0.2,setosa,21.16
4,5.0,3.6,1.4,0.2,setosa,25.00


Transformation involving multiple columns. Uses `axis=1` to access columns.  
Compute average of `sepal_length`, `sepal_width`:

In [6]:
iris['sepal_len_wid_avg'] = iris[['sepal_length','sepal_width']].apply(lambda x: (x.sepal_length+x.sepal_width)/2, axis=1)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_len_sq,sepal_len_wid_avg
0,5.1,3.5,1.4,0.2,setosa,26.01,4.30
1,4.9,3.0,1.4,0.2,setosa,24.01,3.95
2,4.7,3.2,1.3,0.2,setosa,22.09,3.95
3,4.6,3.1,1.5,0.2,setosa,21.16,3.85
4,5.0,3.6,1.4,0.2,setosa,25.00,4.30


### TRY FOR YOURSELF (UNGRADED EXERCISES)

1) Use `apply()` to append a new column that is the minimum of (petal_length, petal_width)

Print the head, tail of the new dataframe to check things look correct.

In [7]:
iris['min_of_petal'] = iris[['petal_length','petal_width']].apply(lambda x: min(x.petal_length, x.petal_width), axis = 1)

In [8]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_len_sq,sepal_len_wid_avg,min_of_petal
0,5.1,3.5,1.4,0.2,setosa,26.01,4.30,0.2
1,4.9,3.0,1.4,0.2,setosa,24.01,3.95,0.2
2,4.7,3.2,1.3,0.2,setosa,22.09,3.95,0.2
3,4.6,3.1,1.5,0.2,setosa,21.16,3.85,0.2
4,5.0,3.6,1.4,0.2,setosa,25.00,4.30,0.2


In [9]:
iris.tail()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_len_sq,sepal_len_wid_avg,min_of_petal
145,6.7,3.0,5.2,2.3,virginica,44.89,4.85,2.3
146,6.3,2.5,5.0,1.9,virginica,39.69,4.40,1.9
147,6.5,3.0,5.2,2.0,virginica,42.25,4.75,2.0
148,6.2,3.4,5.4,2.3,virginica,38.44,4.80,2.3
149,5.9,3.0,5.1,1.8,virginica,34.81,4.45,1.8


In [ ]:
iris['min_petal']=iris[['petal_length','petal_width']].apply(lambda x: min(x.petal_length, x.petal_width), axis=1)
print(iris.head())
print(iris.tail())

## II. Aggregation

Involves one or more of:

- splitting the data into groups
- applying a function to each group
- combining results

### `groupby()`

Compute mean of each column, grouped (separately) by species

In [12]:
iris.groupby("species").mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


### `pivot_table()`

Apply a function `aggfunc` to selected values grouped by columns

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

Compute mean sepal length for each species:

In [13]:
pd.pivot_table(iris, values="sepal_length", columns=["species"], aggfunc = np.mean)

/var/folders/40/rct_612d1zd0wb7f62jvwlk40000gn/T/ipykernel_74981/3739325787.py:1: FutureWarning: The provided callable <function mean at 0x1049260c0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.pivot_table(iris, values="sepal_length", columns=["species"], aggfunc = np.mean)


species,setosa,versicolor,virginica
sepal_length,5.006,5.936,6.588


In [14]:
?pd.pivot_table

Signature:
pd.pivot_table(
    data: 'DataFrame',
    values=None,
    index=None,
    columns=None,
    aggfunc: 'AggFuncType' = 'mean',
    fill_value=None,
    margins: 'bool' = False,
    dropna: 'bool' = True,
    margins_name: 'Hashable' = 'All',
    observed: 'bool | lib.NoDefault' = <no_default>,
    sort: 'bool' = True,
) -> 'DataFrame'
Docstring:
Create a spreadsheet-style pivot table as a DataFrame.

The levels in the pivot table will be stored in MultiIndex objects
(hierarchical indexes) on the index and columns of the result DataFrame.

Parameters
----------
data : DataFrame
values : list-like or scalar, optional
    Column or columns to aggregate.
index : column, Grouper, array, or list of the previous
    Keys to group by on the pivot table index. If a list is passed,
    it can contain any of the other types (except list). If an array is
    passed, it must be the same length as the data and will be used in
    the same manner as column values.
columns : column, Grouper

In [15]:
pd.pivot_table(iris, values="sepal_length", columns=["species"], aggfunc = "mean")
# Since the pandas package has been updated, the aggfunc needs a "" rather than np.mean)

species,setosa,versicolor,virginica
sepal_length,5.006,5.936,6.588


### TRY FOR YOURSELF (UNGRADED EXERCISES)

2) Use a pivot table to compute the following statistics on sepal_width and petal_width grouped by species:

- median  
- mean

These can be computed together in a single call to `pivot_table()`.

In [16]:
pd.pivot_table(iris, values=["sepal_width", "petal_width"], columns=['species'], aggfunc={'median', 'mean'})

species             setosa  versicolor  virginica
petal_width mean     0.246       1.326      2.026
            median   0.200       1.300      2.000
sepal_width mean     3.428       2.770      2.974
            median   3.400       2.800      3.000

In [17]:
pd.pivot_table(iris, values=["sepal_width","petal_width"], columns=["species"], aggfunc = {np.mean, np.median})

/var/folders/40/rct_612d1zd0wb7f62jvwlk40000gn/T/ipykernel_74981/2238371434.py:1: FutureWarning: The provided callable <function median at 0x104a5cd60> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  pd.pivot_table(iris, values=["sepal_width","petal_width"], columns=["species"], aggfunc = {np.mean, np.median})
/var/folders/40/rct_612d1zd0wb7f62jvwlk40000gn/T/ipykernel_74981/2238371434.py:1: FutureWarning: The provided callable <function mean at 0x1049260c0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.pivot_table(iris, values=["sepal_width","petal_width"], columns=["species"], aggfunc = {np.mean, np.median})
/var/folders/40/rct_612d1zd0wb7f62jvwlk40000gn/T/ipykernel_74981/2238371434.py:1: FutureWarning: The provided callable <fu

species             setosa  versicolor  virginica
petal_width mean     0.246       1.326      2.026
            median   0.200       1.300      2.000
sepal_width mean     3.428       2.770      2.974
            median   3.400       2.800      3.000

## III. Merging/Joining DataFrames


### `concat()`  

Concatenate pandas objects along an axis

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

Create two dfs and vertically stack them

In [18]:
df1 = pd.DataFrame(np.random.randn(3, 4))
df2 = pd.DataFrame(np.random.randn(3, 4))

print(df1)
print('-'*45)
print(df2)

df3 = pd.concat([df1, df2], axis=0) # axis = 0 means that the table is gonna be combined together in column side.

print('-'*45)
print(df3)

df4 = pd.concat([df1, df2], axis=1) # axis = 1 means that the table is gonna be combined together in row side.

print('-'*45)
print(df4)

          0         1         2         3
0 -0.913411  0.154209 -0.189337 -0.570711
1 -0.369618  0.796787 -0.700786  1.140073
2 -1.514481 -0.860250  2.079876 -0.685243
---------------------------------------------
          0         1         2         3
0 -0.687458  0.190347 -0.431969 -0.157629
1 -0.273177 -0.185986  1.717933 -0.092401
2  1.028273  0.662249 -2.749386 -0.234200
---------------------------------------------
          0         1         2         3
0 -0.913411  0.154209 -0.189337 -0.570711
1 -0.369618  0.796787 -0.700786  1.140073
2 -1.514481 -0.860250  2.079876 -0.685243
0 -0.687458  0.190347 -0.431969 -0.157629
1 -0.273177 -0.185986  1.717933 -0.092401
2  1.028273  0.662249 -2.749386 -0.234200
---------------------------------------------
          0         1         2         3         0         1         2  \
0 -0.913411  0.154209 -0.189337 -0.570711 -0.687458  0.190347 -0.431969   
1 -0.369618  0.796787 -0.700786  1.140073 -0.273177 -0.185986  1.717933   
2 -1.51

### `merge()`

SQL-style joining of tables (DataFrames)

Important parameters include:

- `how` : type of merge {'left', 'right', 'outer', 'inner', 'cross'}, default ‘inner’
- `on`  : names to join on
        
[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

Create two tables, `left` and `right`. Then right join them on `key`.  
Right join means include all records from table on right.  
The `key` is used for matching up the records.

In [19]:
left = pd.DataFrame({"key": ["jamie", "bill"], "lval": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "rval": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

---left
     key  lval
0  jamie    15
1   bill    22

---right
     key  rval
0  jamie     4
1   bill     5
2  asher     8

---joined
     key  lval  rval
0  jamie  15.0     4
1   bill  22.0     5
2  asher   NaN     8


Notice the NaN inserted into the record with key=asher, since the left table didn't contain the key.

**Matching column names**  
In this next example, the value columns have the same name: *val*.  Notice what happens to the column names.

In [20]:
left = pd.DataFrame({"key": ["jamie", "bill"], "val": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "val": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

---left
     key  val
0  jamie   15
1   bill   22

---right
     key  val
0  jamie    4
1   bill    5
2  asher    8

---joined
     key  val_x  val_y
0  jamie   15.0      4
1   bill   22.0      5
2  asher    NaN      8


### TRY FOR YOURSELF (UNGRADED EXERCISES)

3) Redo the join exercise above, using an inner join instead of a right join.  
Make sure the results make sense.

In [ ]:
left = pd.DataFrame({"key": ["jamie", "bill"], "lval": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "rval": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="inner")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

add solution and fold to hide

## IV. Reshape

Changes the object's shape

We illustrate creating pandas Series, extracting array of length 6, and reshaping to 3x2 array.

In [21]:
# create a series 
ser = pd.Series([1, 1, 2, 3, 5, 8]) 

# extract values 
vals = ser.values 

print('orig data:', vals)
print('orig type:', type(vals))
print('orig shape:', vals.shape)

# reshaping series
reshaped_vals = vals.reshape((3, 2)) 

print('\n reshaped vals:')
print(reshaped_vals)
print('\n new type:', type(reshaped_vals))
print('new shape:', reshaped_vals.shape)

orig data: [1 1 2 3 5 8]
orig type: <class 'numpy.ndarray'>
orig shape: (6,)

 reshaped vals:
[[1 1]
 [2 3]
 [5 8]]

 new type: <class 'numpy.ndarray'>
new shape: (3, 2)


Including -1 as one of the dimensions tells numpy: infer this dimension from the data and the other dimensions.

Example: enforce 3 columns:

In [22]:
vals.reshape(-1,3)

array([[1, 1, 2],
       [3, 5, 8]])

Enforce 3 rows:

In [23]:
vals.reshape(3,-1)

array([[1, 1],
       [2, 3],
       [5, 8]])

**IMPORTANT NOTE**  

Notice the shape of original array: `(6,)`  
This is a vector with one dimension, and is different from two-dimensional `(6,1)` array

### TRY FOR YOURSELF (UNGRADED EXERCISES)

4) Recreate the series from above with data [1, 1, 2, 3, 5, 8]  
Extract the data from the series and reshape to 2x3.  
Print both the reshaped data, and the dimensions.

In [ ]:
ser = pd.Series([1, 1, 2, 3, 5, 8])
vals= ser.values
resh = vals.reshape(2,3)
print(resh)
print('new shape:', resh.shape)

## V. Categoricals

Categorical data takes discrete values where computation on the values does not make sense.  
Zip code is a typical example.

To include categoricals in models, they must be converted to numeric.  

### `get_dummies()`
Dummy code categorical data

Important parameters: 

- `prefix`    : append prefix to column names (a good idea for later use)
- `drop_first`: remove first level, as only `k-1` variables needed to represent `k` levels

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

In [25]:
cats = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese']})

print('--categorical data')
print(cats)

cats = pd.get_dummies(cats.breed, drop_first=True, prefix='breed') # you can also drop the "drop_first=Ture"

print('\n')
print('--dummified categorical data')
print(cats)

--categorical data
       breed
0    persian
1    persian
2    siamese
3  himalayan
4    burmese


--dummified categorical data
   breed_himalayan  breed_persian  breed_siamese
0            False           True          False
1            False           True          False
2            False          False           True
3             True          False          False
4            False          False          False


Notice `burmese` was dropped (first level by alphabet)

### TRY FOR YOURSELF (UNGRADED EXERCISES)

5) The dataframe below contains two categoricals. Dummify each of them, giving them a prefix and dropping the first level from each. 

Print the new dataframe to insure correctness.

Hint: You might want to dummify each column into separate new dataframes, and then merge them together by using:

`pd.concat([df1, df2], axis=1)`

In [ ]:
cats2 = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese'], 
                      'color':['calico','white','seal point','cream','sable']})

In [ ]:
cats2 = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese'], 
                      'color':['calico','white','seal point','cream','sable']})

breed = pd.get_dummies(cats2.breed, drop_first=True, prefix='breed_')
color = pd.get_dummies(cats2.color, drop_first=True, prefix='color_')

cats3 = pd.concat([breed, color], axis=1)
cats3

---